In [41]:
import pandas as pd
import csv
import json

In [42]:
height = 730
width = 600

fields = []

In [45]:
def generateDf(filename):
    jsonObj = json.load(open(filename, 'r'))
    newJson = jsonObj[2]['Label']['objects']
    df = pd.DataFrame(columns=['top','left','height','width','type'])
    for data in newJson:
        tmp = data['bbox']
        tmp['type']=data['value']
        df = df.append(tmp,ignore_index=True)
    df['group']='NaN'
    df['value']='NaN'
    return df

df = generateDf('exported_json.json')
df

,top,left,height,width,type,group,value
0,362,1451,65,131,label,NaN,NaN
1,187,1068,83,401,label,NaN,NaN
2,580,292,56,148,label,NaN,NaN
3,741,292,69,161,label,NaN,NaN
4,749,1151,61,139,label,NaN,NaN
5,950,301,57,1041,label,NaN,NaN
6,1072,445,70,104,label,NaN,NaN
7,1081,724,56,82,label,NaN,NaN
8,1089,1002,61,258,label,NaN,NaN
9,1220,288,70,257,label,NaN,NaN


In [ ]:
structure = -1   # 0: horizontal (Fields are beside labels)  1: vertical  -1: not set
i = 0; j=0


In [ ]:
def isFieldInRight(i,j):
    if(labelsdf['left'][i]+labelsdf['width'][i] <= fieldsdf['left'][j]):
        if(fieldsdf['top'][j]+fieldsdf['height'][j] >= labelsdf['top'][i]+labelsdf['height'][i] and fieldsdf['top'][j] <= labelsdf['top'][i]+labelsdf['height'][i]):
            # field is found at the right side with proper place
            return True
    return False

def isFieldBelow(i,j):
    if(labelsdf['top'][i]+labelsdf['height'][i] <= fieldsdf['top'][j]):
        if(fieldsdf['left'][j] <= labelsdf['left'][i] and fieldsdf['left'][j]+fieldsdf['width'][j] >= labelsdf['left'][i]+labelsdf['width'][i]):
            # field is found below 
            return True
    return False

def isLabelInRight(i):
    if(i<labelsdf.count-1):
        if(labelsdf['left'][i]+labelsdf['width'][i] < labelsdf['left'][i+1]):
            if(labelsdf['top'][i]+labelsdf['height'][i] >= labelsdf['top'][i+1] and labelsdf['top'][i+1]+labelsdf['height'][i+1] > labelsdf['top'][i]):
                # label is found at the right side with proper place
                return True
    return False
    
def isLabelBelow(i):
    if(i<labelsdf.count-1):
        if(labelsdf['top'][i]+labelsdf['height'][i] < labelsdf['top'][i+1]):
            if(labelsdf['left'][i] <= labelsdf['left'][i+1]+labelsdf['width'][i+1] and labelsdf['left'][i]+labelsdf['width'][i] > labelsdf['left'][i+1]):
                # field is found below 
                return True
    return False
    

In [10]:
def decideStructure(i,j):
    if(isFieldInRight(i,j) and isLabelBelow(i,j)):
        structure=0
    if(isLabelBelow(i,j) and isFieldInRight(i,j)):
        structure=1

In [11]:
def elementsInRows(i,j):
    labels=0;fields=0
    while(labelsdf['left'][i]+labelsdf['width'][i]<600):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-11-c6675d75f021>, line 4)

In [13]:
#ERROR   ( max_field_height - current_height )/2  strip_top = current_top - ERROR   strip_bottm = current_bottom + ERRROR

In [68]:
df = df.sort_values(by=['top','left']).reset_index(drop=True)
df

,top,left,height,width,type,group,value
0,187,1068,83,401,label,NaN,NaN
1,344,1665,96,614,field,NaN,NaN
2,362,1451,65,131,label,NaN,NaN
3,536,501,122,1351,field,NaN,NaN
4,580,292,56,148,label,NaN,NaN
5,715,1316,104,854,field,NaN,NaN
6,719,497,100,606,field,NaN,NaN
7,741,292,69,161,label,NaN,NaN
8,749,1151,61,139,label,NaN,NaN
9,950,301,57,1041,label,NaN,NaN


In [50]:
min_field_height = df['height'].min()
print(min_field_height)
max_field_height = df['height'].max()
print(max_field_height)

56
213


In [67]:
element=-1
print(df.shape[0])
while(element < df.shape[0]):
    element+=1;labels=0;fields=0
    topy=0;bottomy=0
    ERROR = (max_field_height - df['height'][element] )/2
    topy=df['top'][element] - ERROR
    bottomy=df['height'][element]+df['top'][element] + ERROR
    print('top:',topy,'bottom:',bottomy)
    curr_df = df[(df.top>=topy) & (df.top+df.height<=bottomy)]
    curr_df.sort_values(by=['left']).reset_index(drop=True)
    print(curr_df)
    for i in range(curr_df.shape[0]):
        element+=1
#         print(curr_df.iloc[i].to_dict)

22
top: 122.0 bottom: 335.0
   top  left height width   type group value
0  187  1068     83   401  label   NaN   NaN
top: 288.0 bottom: 501.0
   top  left height width   type group value
1  344  1665     96   614  field   NaN   NaN
2  362  1451     65   131  label   NaN   NaN
top: 660.5 bottom: 873.5
   top  left height width   type group value
5  715  1316    104   854  field   NaN   NaN
6  719   497    100   606  field   NaN   NaN
7  741   292     69   161  label   NaN   NaN
8  749  1151     61   139  label   NaN   NaN
top: 998.0 bottom: 1211.0
     top  left height width      type group value
10  1059   305     91   101  checkbox   NaN   NaN
11  1072   445     70   104     label   NaN   NaN
12  1072   854     70    92  checkbox   NaN   NaN
13  1081   724     56    82     label   NaN   NaN
14  1085   584     57    96  checkbox   NaN   NaN
15  1089  1002     61   258     label   NaN   NaN
top: 1325.0 bottom: 1538.0
     top left height width   type group value
17  1338  301    187  1